In [27]:
# Allow multiple displays per cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [28]:
%load_ext dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [29]:
import os

In [30]:
!pwd
!ls -l ./

/datadrive01/prj/AzureChestXRayNoAML/code
total 692
-rwxrwxr-x 1 loginvm001 loginvm001 409251 Jan 26 03:32 00_create_docker_image.html
-rwxrwxr-x 1 loginvm001 loginvm001  15270 Jan 26 03:32 00_create_docker_image.ipynb
drwxrwxr-x 2 loginvm001 loginvm001   4096 Jan 24 17:52 01_DataPrep
drwxrwxr-x 2 loginvm001 loginvm001   4096 Jan 24 17:52 02_Model
drwxrwxr-x 2 loginvm001 loginvm001   4096 Jan 24 17:52 docker_history
-rwxrwxr-x 1 loginvm001 loginvm001 256265 Jan 24 17:52 edit_python_files.html
-rwxrwxr-x 1 loginvm001 loginvm001   6867 Jan 24 17:52 edit_python_files.ipynb
drwxrwxr-x 2 loginvm001 loginvm001   4096 Jan 24 17:52 src


In [41]:
%%writefile .env
# This cell is tagged `parameters`
# Please modify the values below as you see fit

# Your docker login and image repository name
docker_login = 'georgedockeraccount'
image_tag = "/chestxray-no-aml-gpu-root-account"
image_version = '1.0.2'

Overwriting .env


In [40]:
%dotenv
os.getenv('image_version')
dockerfile_name = 'dockerfile'+ '_' + os.getenv('image_version')

'1.0.1'

In [33]:
%%writefile ./../docker/$dockerfile_name

FROM nvidia/cuda:9.0-cudnn7-devel
#FROM nvidia/cuda:9.0-cudnn7-devel-ubuntu16.04

MAINTAINER George Iordanescu <ghiordan@microsoft.com>
#based on https://github.com/keras-team/keras/blob/master/docker/Dockerfile

# Install system packages
RUN apt-get update && apt-get install -y --no-install-recommends \
      apt-utils \
      bzip2 \
      g++ \
      git \
      graphviz \
      libgl1-mesa-glx \
      libhdf5-dev \
      openmpi-bin \
      wget \
      ssh \
      rsync && \
    rm -rf /var/lib/apt/lists/*

# Install conda
ENV CONDA_DIR /opt/conda
ENV PATH $CONDA_DIR/bin:$PATH

RUN wget --quiet --no-check-certificate https://repo.continuum.io/miniconda/Miniconda3-4.2.12-Linux-x86_64.sh && \
    echo "c59b3dd3cad550ac7596e0d599b91e75d88826db132e4146030ef471bb434e9a *Miniconda3-4.2.12-Linux-x86_64.sh" | sha256sum -c - && \
    /bin/bash /Miniconda3-4.2.12-Linux-x86_64.sh -f -b -p $CONDA_DIR && \
    rm Miniconda3-4.2.12-Linux-x86_64.sh && \
    echo export PATH=$CONDA_DIR/bin:'$PATH' > /etc/profile.d/conda.sh 
        
#https://docs.microsoft.com/en-us/azure/storage/common/storage-use-azcopy-linux
# https://docs.microsoft.com/en-us/azure/storage/common/storage-use-azcopy-linux#download-and-install-azcopy
# https://www.microsoft.com/net/download/linux-package-manager/ubuntu16-04/runtime-2.1.2
RUN wget -q https://packages.microsoft.com/config/ubuntu/16.04/packages-microsoft-prod.deb && \
    dpkg -i packages-microsoft-prod.deb && \ 
    apt-get install  -y --no-install-recommends apt-transport-https && \
    apt-get update && \
    apt-get install  -y --no-install-recommends aspnetcore-runtime-2.1  && \
    mkdir /tmp/azcopy && \
    wget -O /tmp/azcopy/azcopy.tar.gz https://aka.ms/downloadazcopyprlinux &&  \
    tar -xf /tmp/azcopy/azcopy.tar.gz -C /tmp/azcopy &&  \
    /tmp/azcopy/install.sh && \
    rm -rf /tmp/azcopy

# Install Python packages and keras
# ENV NB_USER keras
# ENV NB_UID 1000

# RUN useradd -m -s /bin/bash -N -u $NB_UID $NB_USER && \
#     chown $NB_USER $CONDA_DIR -R && \
#     mkdir -p /src && \
#     chown $NB_USER /src

# USER $NB_USER
RUN mkdir -p /sr

ARG python_version=3.6

# tensorflow-gpu 1.10.1 has requirement numpy<=1.14.5,>=1.13.3, but you'll have numpy 1.15.2 which is incompatible.
RUN conda install -y python=${python_version} && \
    pip install --upgrade pip && \
    pip install \
      sklearn_pandas \
      scikit-image \
      tensorflow-gpu \
      tensorflow-tensorboard \
      matplotlib \
      numpy==1.14.5 && \
    conda install \
      bcolz \
      h5py \
      matplotlib \
      mkl \
      nose \
      notebook=5.6.0 \
      opencv \
      Pillow \
      pandas \
      pydot \
      pygpu \
      pyyaml \
      requests \
      scikit-learn \
      six \
      tqdm && \
    conda install -c conda-forge imageio && \
    git clone git://github.com/keras-team/keras.git /src && pip install -e /src[tests] && \
    pip install git+git://github.com/keras-team/keras.git && \
    pip install git+https://github.com/aleju/imgaug && \
    pip install git+https://www.github.com/keras-team/keras-contrib.git && \
    pip install http://download.pytorch.org/whl/cu90/torch-0.4.0-cp36-cp36m-linux_x86_64.whl \
      torchvision && \
    conda clean -yt     

#ADD theanorc /home/keras/.theanorc

ENV PYTHONPATH='/src/:$PYTHONPATH'

WORKDIR /src

EXPOSE 8888

# CMD jupyter notebook --port=8888 --ip=0.0.0.0


Overwriting ./../docker/dockerfile_1.0.1


In [34]:
%dotenv
docker_image_name = os.getenv('docker_login') + os.getenv('image_tag') + ':' + os.getenv('image_version')
docker_file_location = os.path.join(*(['.', '..','docker', dockerfile_name]))
working_path = '.'

docker_image_name
docker_file_location

'georgedockeraccount/chestxray-no-aml-gpu-root-account:1.0.1'

'./../docker/dockerfile_1.0.1'

In [35]:
!mkdir -p ./docker_history
!cp $docker_file_location ./docker_history/

In [36]:
!echo docker build -t $docker_image_name -f $docker_file_location $working_path
!docker build -t $docker_image_name -f $docker_file_location $working_path

docker build -t georgedockeraccount/chestxray-no-aml-gpu-root-account:1.0.1 -f ./../docker/dockerfile_1.0.1 .
Sending build context to Docker daemon  5.023MB
Step 1/13 : FROM nvidia/cuda:9.0-cudnn7-devel
 ---> afc5ab1e9a0d
Step 2/13 : MAINTAINER George Iordanescu <ghiordan@microsoft.com>
 ---> Using cache
 ---> 69afd8783ecc
Step 3/13 : RUN apt-get update && apt-get install -y --no-install-recommends       apt-utils       bzip2       g++       git       graphviz       libgl1-mesa-glx       libhdf5-dev       openmpi-bin       wget       ssh       rsync &&     rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> 934e4d56a2f5
Step 4/13 : ENV CONDA_DIR /opt/conda
 ---> Using cache
 ---> eeb8740afbf1
Step 5/13 : ENV PATH $CONDA_DIR/bin:$PATH
 ---> Using cache
 ---> 2cecb2d2db98
Step 6/13 : RUN wget --quiet --no-check-certificate https://repo.continuum.io/miniconda/Miniconda3-4.2.12-Linux-x86_64.sh &&     echo "c59b3dd3cad550ac7596e0d599b91e75d88826db132e4146030ef471bb434e9a *Miniconda3-4.2.12

In [37]:
!docker push $docker_image_name

The push refers to repository [docker.io/georgedockeraccount/chestxray-no-aml-gpu-root-account]

4dd5d846: Preparing 
500cdad0: Preparing 
d9c85619: Preparing 
08bfc7f5: Preparing 
255199c7: Preparing 
39f3990e: Preparing 
347e5346: Preparing 
9e5e9701: Preparing 
51016399: Preparing 
b658aaf0: Preparing 
a9c7c79e: Preparing 
a8049aa6: Preparing 
9c0f8a0b: Preparing 
8ccd260b: Preparing 
91f0ffec: Layer already exists 1.0.1: digest: sha256:21b40a4a52af54931a57a00af6d25ecdbbcd75cff48aa0e9137b261e7640ce31 size: 3475


In [38]:
!echo nvidia-docker run -i -t -p 10003:8888 -v '$(pwd)':/local_dir:rw $docker_image_name /bin/bash -c '"/opt/conda/bin/jupyter notebook --notebook-dir=/local_dir --ip='*' --port=8888 --no-browser --allow-root"'

nvidia-docker run -i -t -p 10003:8888 -v $(pwd):/local_dir:rw georgedockeraccount/chestxray-no-aml-gpu-root-account:1.0.1 /bin/bash -c "/opt/conda/bin/jupyter notebook --notebook-dir=/local_dir --ip=* --port=8888 --no-browser --allow-root"


In [39]:
!jupyter nbconvert --to html 00_create_docker_image.ipynb

[NbConvertApp] Converting notebook 00_create_docker_image.ipynb to html
[NbConvertApp] Writing 267620 bytes to 00_create_docker_image.html
